[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/fast/fast_stable_diffusion.ipynb)


https://github.com/Van-wise/sd-colab


**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244

In [ ]:
#@title 👇 首次运行！
import os
import sys
import binascii
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w = binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
sai = binascii.a2b_uu("=<W1A8FQE+61I9F9U<VEO;BUS=&%B:6QI='DM86D`").decode('utf-8')
sd = binascii.a2b_uu("04W1A8FQE+61I9F9U<VEO;@``").decode('utf-8')

sd_dir = '/root/main'
gsd_dir = f'/content/drive/MyDrive/main'

#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

# git clone AUTOMATIC1111
!apt -y install -qq aria2 git
%cd /content
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  !git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw /root/main

# Requirements
with capture.capture_output() as cap:
  %cd /content
  !mkdir -p {sd_dir}/repositories/{sai}
  !wget -q -i https://raw.githubusercontent.com/Van-wise/fast-sd/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content/ --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  !mv -n /content/sd/stablediffusion/* {sd_dir}/repositories/{sai} && rm -rf /content/sd
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

!apt --fix-broken install
!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv image-reward pyfunctional

clear_output()
inf('\u2714 Done','success', '50px')

#安装中文&必备插件
!git clone --depth 1 https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {sd_dir}/extensions/$sdw-localization-zh_Hans #中文
!git clone --depth 1 https://github.com/zanllp/$w-infinite-image-browsing {sd_dir}/extensions/$w-infinite-image-browsing #图片浏览
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {sd_dir}/extensions/$w-infinite-image-browsing -o .env
!git clone --depth 1 https://github.com/etherealxx/batchlinks-webui {sd_dir}/extensions/batchlinks-webui #快捷下载
!git clone --depth 1 https://github.com/Mikubill/$w-controlnet {sd_dir}/extensions/$w-controlnet #controlnet
!git clone --depth 1 https://github.com/Physton/$w-prompt-all-in-one {sd_dir}/extensions/$w-prompt-all-in-one #提示词

use_roop  = True  #@param {type:"boolean"}
if use_roop:
  !git clone --depth 1 https://github.com/s0md3v/$w-roop {sd_dir}/extensions/$w-roop #roop换脸插件
  !pip install insightface==0.7.3
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -d {sd_dir}/models/roop -o inswapper_128.onnx

#负面提示词
!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {sd_dir}/embeddings && rm -rf /content/negative
!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {sd_dir}/models/Lora && rm -rf /content/lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {sd_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#同步 秋葉aaaki Webui (可选)
%cd {sd_dir}
config_link = "https://raw.githubusercontent.com/Van-wise/sd-colab/main/config.json" #@param {type:"string"}"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {sd_dir} -o config.json

clear_output()
inf('\u2714 Done','success', '50px')

#CKPT Model Download/Load
model_dir = f"{sd_dir}/models/{sd}"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors
#vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {sd_dir}/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors
#Lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/62833 -d {sd_dir}/models/Lora -o Detail_Tweaker_LoRA.safetensors ##Detail Tweaker LoRA/

clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')

#链接 controlnet
#@markdown 选择ControlNet：
#@markdown
#@markdown 0：不需要使用ControlNet
#@markdown
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()
inf('\u2714 Done','success', '50px')

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {sd_dir}/modules
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {sd_dir}
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {sd_dir}/webui.py
    !sed -i 's@print(\"No module.*@@' {sd_dir}/repositories/{sai}/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {sd_dir}/modules/shared.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"{sd_dir}/repositories/{sai}\"]@' {sd_dir}/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' {sd_dir}/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' {sd_dir}/modules/paths.py


wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --theme dark --upcast-sampling"
!python {sd_dir}/webui.py $wise


In [ ]:
#@title 👇启动 Stable-Diffusion
from IPython.utils import capture
import sys


wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --theme dark"
!python {sd_dir}/webui.py $wise


In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {sd_dir}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {sd_dir}/models/Lora/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {sd_dir}/models/Lora -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gsd_dir}/models/Lora"  # f"{gsd_dir}/models/Lora/Stable-diffusion/"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}